In [ ]:
# Setup code for Google Colab
USE_COLAB = True
GOOGLE_DRIVE_PATH = ''
if USE_COLAB:
    print("Using Colab!")
    %load_ext autoreload
    %autoreload 2

    from google.colab import drive
    drive.mount('/content/drive')

    import os
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'EECS545/SVM-Variations'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))

    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)

    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torchvision
    import statistics
    import random
    import time
    import math
    import numpy as np
    import cv2
    import copy
    import shutil
    import os
    import json

    import matplotlib.pyplot as plt
    %matplotlib inline

    !pip3 install -q idx2numpy
    
    !pip3 install -q cvxopt

    # install qp solver
    !pip3 install -q qpth
    import qpth

    if torch.cuda.is_available:
      print('Good to go!')
    else:
      print('Please set GPU via Edit -> Notebook Settings.')


In [ ]:
import numpy as np
import torch
import scipy.io as sio
import matplotlib.pyplot as plt

# import SVMClass

np.random.seed(0)

############
# load and plot data
############
if USE_COLAB:
    nuclear = sio.loadmat(os.path.join(GOOGLE_DRIVE_PATH,'Data/nuclear.mat'))
else:
    nuclear = sio.loadmat('Data/nuclear.mat')

x = torch.tensor(nuclear['x'], dtype = torch.float32, device = 'cuda')
y = torch.tensor(nuclear['y'], dtype = torch.long, device = 'cuda')

N = x.shape[1]

## reshape data
x = x.t()               # x \in (N,D)
y = y.reshape(N)        # y \in (N,)

# plot the data:
negInd = y == -1
posInd = y == 1
plt.scatter(x.cpu()[negInd, 0], x.cpu()[negInd, 1], color='b')
plt.scatter(x.cpu()[posInd, 0], x.cpu()[posInd, 1], color='r')
plt.figure(1)
plt.show()

# change the label into {0,1}
# y = (y+1)//2

# Preprocess: append 1s at the end of data vectors
ones_x = torch.ones((N,1), dtype = torch.float32, device = 'cuda')
x_with_ones = torch.cat((x,ones_x),axis=1)
x = x_with_ones



# Verifying with EECS545 HW3b

In [ ]:
##########################
# get accuracy of sk lib svm implementation and our accuracy
#########################
from LinearSVM import *

N = x.shape[0]
indices = torch.randperm(N)
train_N = N # 3 quarters of data is training. 
x_train = x[indices[0:train_N]]
y_train = y[indices[0:train_N]]
x_test = x[indices[train_N:]]
y_test = y[indices[train_N:]]

apply_sklearn_svm(x_train, y_train, x_test, y_test, max_iters = 1000)

loss_history, LSVM = apply_LSVM(x_train, y_train, x_test, y_test, max_iters = 1000)



##############################
# plot our classifier line and Loss
#############################
W = LSVM.W[:]

w1 = W[0].cpu()
w2 = W[1].cpu()
b = W[2].cpu()

x0_vals = torch.tensor(np.linspace(0, 8, 8)).reshape(1,-1)
y_vals = (-b - w1*x0_vals) / w2

plt.figure(1)
plt.plot(x0_vals[0,:], y_vals[0,:], color='black', label='Learned line')
plt.title('data')
# plt.show()

negInd = y == -1
posInd = y == 1
plt.scatter(x.cpu()[negInd, 0], x.cpu()[negInd, 1], color='b')
plt.scatter(x.cpu()[posInd, 0], x.cpu()[posInd, 1], color='r')
plt.figure(1)
plt.show()

plt.figure(2)
plt.plot(loss_history.cpu().detach().numpy())
plt.title('Objective function J')
plt.show()

In [ ]:
# Search for hyperparameters that yield max accuracy
model = LinearSVM()
learning_rates = [1e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 2]
regs = [1e-4, 1e-3, 5e-3, 1e-2, 1e-1]
folds = 10

search_for_hyperparams(model, x_train, y_train, folds, learning_rates, regs)



In [ ]:
# Mnist Test Code
from LinearSVM import *
import Mnist_loader as mnist
x_train, y_train, x_test, y_test = mnist.load_odd_even_Mnist()
x_train, x_test, mu, std = mnist.preprocess_Mnist(x_train, x_test)
apply_LSVM(x_train, y_train, x_test, y_test, max_iters = 1000)
apply_sklearn_svm(x_train, y_train, x_test, y_test, max_iters = 1000)

In [ ]:
# SVMGuide1 Test Code
USE_COLAB = False
GOOGLE_DRIVE_PATH = ''
from LinearSVM import *
from SVMGuide_loader import load_SVMGuide1
x_train, y_train, x_test, y_test, mu, std = load_SVMGuide1(USE_COLAB, GOOGLE_DRIVE_PATH)
apply_LSVM(x_train, y_train, x_test, y_test, max_iters = 1000)
apply_sklearn_svm(x_train, y_train, x_test, y_test, max_iters = 1000)


In [ ]:
# Using KDD Dataset Test Code:
# Download dataset from our google drive first. https://drive.google.com/drive/u/3/folders/1Alq22QmrDgMPhMfJVh-5WmgrKJmzTqCz
from KDD_loader import kdd_load
x_train, y_train, x_test, y_test = kdd_load()

from LinearSVM import *
from Evaluate_models import *
LSVM = LinearSVM() 
LSVM.__init__()
time, acc = evaluate_model(LSVM, x_train, y_train, x_test, y_test)
print(time, acc)
# apply_sklearn_svm(x_train, y_train, x_test, y_test, max_iters = 1000)

In [ ]:
# Code to compare different models in time and accuracy:
USE_COLAB = False
GOOGLE_DRIVE_PATH = ''
from LinearSVM import *
from KernelSVM import *
from KmeanSVM import *
from ClusterSVM import *
from Evaluate_models import *

# initialize hyperparameters: corresponding to mnist oe, mnist 38, svmguide1, kdd
C = [1e3] * 4
lamb = [0.01] * 4
K = [2, 2, 4, 4]
reg = [1e-7, 1e-5, 1e-5, 1e-6]
lr = [50, 50, 1, 0.1]


# define models:
LSVM = LinearSVM()  
KSVM_sk = KernelSVM_sklearn()
KSVM = kernelSVM()
KMSVM = KmeanSVM()
CSVM = ClusterSVM()

# evaluate using Mnist:
# LSVM_time_oe, LSVM_acc_oe, LSVM_time_38, LSVM_acc_38 = evaluate_using_mnist(LSVM)
# KSVM_sk_time_oe, KSVM_sk_acc_oe, KSVM_sk_time_38, KSVM_sk_acc_38 = evaluate_using_mnist(KSVM_sk, use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# KSVM_time_oe, KSVM_acc_oe, KSVM_time_38, KSVM_acc_38 = evaluate_using_mnist(KSVM, C = C[0:2], use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# KMSVM_time_oe, KMSVM_acc_oe, KMSVM_time_38, KMSVM_acc_38 = evaluate_using_mnist(KMSVM, K = K[0:2], lamb = lamb[0:2], reg = reg[0:2], learning_rate = lr[0:2], use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# CSVM_time_oe, CSVM_acc_oe, CSVM_time_38, CSVM_acc_38 = evaluate_using_mnist(CSVM)


print("With Mnist odd/even dataset, we get:")
# print("Using LSVM: Time = ", LSVM_time_oe, "  acc = ", LSVM_acc_oe)
# print("Using KSVM_sk: Time = ", KSVM_sk_time_oe, "  acc = ", KSVM_sk_acc_oe)
# print("Using KSVM: Time = ", KSVM_time_oe, "  acc = ", KSVM_acc_oe)
# print("Using KMSVM: Time = ", KMSVM_time_oe, "  acc = ", KMSVM_acc_oe)
# print("Using CSVM: Time = ", CSVM_time_oe, "  acc = ", CSVM_acc_oe)

print("With Mnist 3/8 dataset, we get:")
# print("Using LSVM: Time = ", LSVM_time_38, "  acc = ", LSVM_acc_38)
# print("Using KSVM_sk: Time = ", KSVM_sk_time_38, "  acc = ", KSVM_sk_acc_38)
# print("Using KSVM: Time = ", KSVM_time_38, "  acc = ", KSVM_acc_38)
# print("Using KMSVM: Time = ", KMSVM_time_38, "  acc = ", KMSVM_acc_38)
# print("Using CSVM: Time = ", CSVM_time_38, "  acc = ", CSVM_acc_38)


# evaluate using SVM_Guide1:
# LSVM_time_svmg, LSVM_acc_svmg = evaluate_using_SVM_Guide1(LSVM)
# KSVM_sk_time_svmg, KSVM_sk_acc_svmg = evaluate_using_SVM_Guide1(KSVM_sk, use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# KSVM_time_svmg, KSVM_acc_svmg = evaluate_using_SVM_Guide1(KSVM, C = C[2], use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
KMSVM_time_svmg, KMSVM_acc_svmg = evaluate_using_SVM_Guide1(KMSVM, K = K[2], lamb = lamb[2], reg = reg[2], learning_rate = lr[2], use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# CSVM_time_svmg, CSVM_acc_svmg = evaluate_using_SVM_Guide1(CSVM)

print("With SVM_Guide1 dataset, we get:")
# print("Using LSVM: Time = ", LSVM_time_svmg, "  acc = ", LSVM_acc_svmg)
# print("Using KSVM_sk: Time = ", KSVM_sk_time_svmg, "  acc = ", KSVM_sk_acc_svmg)
# print("Using KSVM: Time = ", KSVM_time_svmg, "  acc = ", KSVM_acc_svmg)
print("Using KMSVM: Time = ", KMSVM_time_svmg, "  acc = ", KMSVM_acc_svmg)
# print("Using CSVM: Time = ", CSVM_time_svmg, "  acc = ", CSVM_acc_svmg)

# evaluate using KDD:
# LSVM_time_kdd, LSVM_acc_kdd = evaluate_using_KDD(LSVM)
# KSVM_sk_time_kdd, KSVM_sk_acc_kdd = evaluate_using_KDD(KSVM_sk, use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# KSVM_time_kdd, KSVM_acc_kdd = evaluate_using_KDD(KSVM, C = C[3], use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# KMSVM_time_kdd, KMSVM_acc_kdd = evaluate_using_KDD(KMSVM, K = K[3], lamb = lamb[3], reg = reg[3], learning_rate = lr[3], use_colab = USE_COLAB, path = GOOGLE_DRIVE_PATH)
# CSVM_time_kdd, CSVM_acc_kdd = evaluate_using_KDD(CSVM)

print("With KDD dataset, we get:")
# print("Using LSVM: Time = ", LSVM_time_kdd, "  acc = ", LSVM_acc_kdd)
# print("Using KSVM_sk: Time = ", KSVM_sk_time_kdd, "  acc = ", KSVM_sk_acc_kdd)
# print("Using KSVM: Time = ", KSVM_time_kdd, "  acc = ", KSVM_acc_kdd)
# print("Using KMSVM: Time = ", KMSVM_time_kdd, "  acc = ", KMSVM_acc_kdd)
# print("Using CSVM: Time = ", CSVM_time_kdd, "  acc = ", CSVM_acc_kdd)

